The task is as follows:
- Download the Credit.csv dataset from ISL: https://www.statlearning.com/s/Credit.csv
- Preprocess the dataset as you see fit.
- Fit non-linear models (GLM, GAM, RF, XGBoost) to predict Balance from the other variables in the dataset. You are not allowed to use the Income variable as a predictor, due to its high collinearity with Balance. Whether you fit one or more types of models is up to you, the same goes for the choice of hyperparameters and how you find them. However, you should aim for achieving good predictive performance, and being relatively certain the performance holds up in new data. In other words, using a train/validation/test-split or a (nested) cross-validation is probably a good idea. When you are satisfied, report model performance in a comprehensible manner.
- Reflect on the choices you made: Why did you choose the model(s) you did? What about the hyperparameters? How did you tune them?
- Reflect upon the performance of the model(s): How well did it/they perform? What would you expect the performance to be on new data? How can you argue this is the case? What other choices could have made you more or less certain about this?

In [2]:
# 1: Load the dataset
import pandas as pd
url = "https://www.statlearning.com/s/Credit.csv"
Credit = pd.read_csv(url)
print(Credit)

      Income  Limit  Rating  Cards  Age  Education  Own Student Married  \
0     14.891   3606     283      2   34         11   No      No     Yes   
1    106.025   6645     483      3   82         15  Yes     Yes     Yes   
2    104.593   7075     514      4   71         11   No      No      No   
3    148.924   9504     681      3   36         11  Yes      No      No   
4     55.882   4897     357      2   68         16   No      No     Yes   
..       ...    ...     ...    ...  ...        ...  ...     ...     ...   
395   12.096   4100     307      3   32         13   No      No     Yes   
396   13.364   3838     296      5   65         17   No      No      No   
397   57.872   4171     321      5   67         12  Yes      No     Yes   
398   37.728   2525     192      1   44         13   No      No     Yes   
399   18.701   5524     415      5   64          7  Yes      No      No   

    Region  Balance  
0    South      333  
1     West      903  
2     West      580  
3     West 